In [1]:
import altair as alt
import pandas as pd
import re

from vega_datasets import data


alt.data_transformers.disable_max_rows()

df = pd.read_csv('EU-UK-Weather-Data.csv')
df.columns

Index(['country', 'location_name', 'latitude', 'longitude',
       'last_updated_epoch', 'temperature_celsius', 'humidity', 'date', 'day',
       'month', 'year', 'week', 'time', 'season', 'condition'],
      dtype='object')

In [2]:
WIDTH, HEIGHT = 500, 600

map_source = alt.topo_feature(data.world_110m.url, 'countries')
map_brush = alt.selection_interval(
    # value={'longitude': [-3, 3], 'latitude': [50, 55]}  # this crops the final chart ?
)
line_chart_scale = alt.selection_interval(bind='scales', encodings=['x'])
city_selection = alt.selection_point(fields=['location_name'], bind='legend')
line_chart_selection = alt.selection_point(fields=['location_name'])
condition_selection = alt.selection_point(fields=['condition'], bind='legend')

base = alt.layer(
    alt.Chart(map_source).mark_geoshape(fill='lightgray', stroke='gray')
).properties(
    width=WIDTH,
    height=HEIGHT,
    projection={
        'type': 'azimuthalEqualArea',
        'rotate': [-10, -52],
        'translate': [WIDTH//2, HEIGHT//2],
        'scale': 700,
        'precision': 0.1
})
city_points = alt.Chart(df).mark_circle(size=50).encode(
    latitude=alt.X('latitude:Q'),
    longitude=alt.Y('longitude:Q'),
    color=alt.condition((city_selection & map_brush & line_chart_selection),
                        alt.Color('location_name:N', title='City',
                                  legend=alt.Legend(orient='left')), alt.value('gray')),
    # opacity=alt.condition(map_brush, alt.value(1), alt.value(0.2)), # does not work on lat long
    tooltip=[alt.Tooltip('location_name', title='City'),
             alt.Tooltip('country', title='Country')],
).properties(
    title='Cities in Europe',
    width=WIDTH,
    height=HEIGHT
).add_params(
    map_brush,
    city_selection,
)

base_points = alt.Chart(df).mark_point(size=1,strokeWidth=5).encode(
    x=alt.X('date:T', title=None, axis=alt.Axis(format='%m-%d')),
    color=alt.Color('location_name:N', legend=None),
    shape=alt.Shape('condition:N', title='Condition', legend=alt.Legend(orient='right')),
).properties(
    width=WIDTH + 250,
    height=0.5 * HEIGHT
).add_params(
    condition_selection
).transform_filter(
    map_brush,
    line_chart_scale,
    city_selection,
    line_chart_selection,
    condition_selection
)
base_line_chart = alt.Chart(df).mark_line(point=False).encode(
    x=alt.X('date:T', title=None, axis=alt.Axis(format='%m-%d')),
    color= alt.Color('location_name:N', legend=None),
    opacity=alt.condition(condition_selection, alt.value(1.0), alt.value(0.1)),
).properties(
    width=WIDTH + 250,
    height=0.5 * HEIGHT
).add_params(
    line_chart_scale,
    line_chart_selection
).transform_filter(
    map_brush,
    line_chart_scale,
    city_selection,
    line_chart_selection,
)
base_line_tooltips = [alt.Tooltip('location_name', title='City'),
                      alt.Tooltip('date:T', title='Date'),
                      alt.Tooltip('condition', title='Condition'),
                      alt.Tooltip('week', title='Week #'),
                      alt.Tooltip('season', title='Season'),]
temp_points = base_points.encode(
    y=alt.Y('temperature_celsius:Q', title='Temperature (°C)'),
    tooltip=base_line_tooltips + [alt.Tooltip('temperature_celsius:Q', title='Temperature °C')]
)
temp_line = base_line_chart.encode(
    y=alt.Y('temperature_celsius:Q', title='Temperature (°C)'),
) + temp_points

humidity_points = base_points.encode(
    y=alt.Y('humidity:Q', title='Humidity %'),
    tooltip=base_line_tooltips + [alt.Tooltip('humidity:Q', title='Humidity (%)')]
)
humidity_line = base_line_chart.encode(
    y=alt.Y('humidity:Q', title='Humidity %'),
) + humidity_points

systemA = ((base + city_points) |
           (temp_line & humidity_line)
          ).properties(
              title='System A'
          )
systemA.save('systemA.html')
systemA

alt.HConcatChart(...)

In [3]:
with open('systemA.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

modified_html = re.sub(
    r'<body>',
    '''
    <body>
    <h2>System A</h2>
    <h3>Usage</h3>
    <ul>
    <li>The city legend can be used as a filter.</li>
    <li>Brushing on the map also filters city locations.</li>
    <li>The condition legend can be used as a filter.</li>
    <li>The temperature and humidity line charts support single line selection.</li>
    <li>The line charts also allow filtering through zooming and panning.</li>
    <li>Refresh the page to reset the system.</li>   
    </ul>
    <h3>Tasks</h3>
    <ol>
    <li>Determine the week # when the first occurrence of a “Snowy” weather condition took place.</li>
    <li>Find three cities that had humidity levels between 40% and 50% in week 28.</li>
    <li>Find the date when London recorded its highest temperature, and determine the lowest temperature recorded across Europe on the same day.</li>
    <li>Rank the cities - Luxembourg, London, and Brussels based on the number of “Fog / Mist” days experienced in November 2024.</li>
    </ol>
    ''',
    html_content
)

with open('systemA.html', 'w', encoding='utf-8') as f:
    f.write(modified_html)